# SpaceX API Data Ingestion and SQL Analysis

In this exercise, you will:

1. Fetch data from the SpaceX API.
2. Store the data in a Python class and perform additional calculations.
3. Store the data in an SQLite database.
4. Write SQL queries and Python functions to interact with the data.

## Step 1: Set Up the Python Class

You will implement a Python class `SpaceXDataHandler` with the following functionality:

1. **Attributes:**
   - `launches`: A list to store launch data.
   - `days_since_last_launch`: A catalog of the days since the last launch for each launch.

2. **Methods:**
   - Fetch data from the SpaceX API endpoints and populate the class attributes.
   - Calculate `days_since_last_launch` for each launch.

Let's implement this class.

In [ ]:
import requests
from datetime import datetime
import sqlite3

class SpaceXDataHandler:
    def __init__(self):
        self.launches = []
        self.days_since_last_launch = {}

    def fetch_launch_data(self):
        """Fetch launch data from the SpaceX API and populate the launches attribute."""
        launches_url = "https://api.spacexdata.com/v4/launches"
        response = requests.get(launches_url)
        response.raise_for_status()
        self.launches = response.json()
        
    def calculate_days_since_last_launch(self):
        """Calculate days since the last launch for each launch."""
        for launch in self.launches:
            launch_date = datetime.fromisoformat(launch['date_utc'].replace('Z', ''))
            days_since = (datetime.utcnow() - launch_date).days
            self.days_since_last_launch[launch['id']] = days_since

# Instantiate the class and fetch the data
spacex_data = SpaceXDataHandler()
spacex_data.fetch_launch_data()
spacex_data.calculate_days_since_last_launch()
print("Days since last launch for each launch (sample):", list(spacex_data.days_since_last_launch.items())[:5])

## Step 2: Store Data in SQLite

You will now store the launch and rocket data into an SQLite database. This will include creating tables and populating them with data from the SpaceX API.

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('spacex_data.db')
c = conn.cursor()

# Create launches table
c.execute('''
    CREATE TABLE IF NOT EXISTS launches (
        launch_id TEXT PRIMARY KEY,
        rocket_id TEXT,
        launch_date TEXT
    )
''')

# Insert launch data into the database
for launch in spacex_data.launches:
    c.execute('''
        INSERT OR IGNORE INTO launches (launch_id, rocket_id, launch_date)
        VALUES (?, ?, ?)
    ''', (launch['id'], launch['rocket'], launch['date_utc']))

conn.commit()
print("Launch data inserted into SQLite database.")

## Step 3: Write a Function to Fetch Crew by Launch ID

Implement a Python function `get_crew_by_launch_id` that queries the SQLite database to retrieve crew members for a given launch ID.

Use the SpaceX API's `crew` endpoint to first populate the crew data.

In [ ]:
# Fetch and store crew data
crew_url = "https://api.spacexdata.com/v4/crew"
crew_response = requests.get(crew_url)
crew_data = crew_response.json()

# Create crew table
c.execute('''
    CREATE TABLE IF NOT EXISTS crew (
        crew_id TEXT PRIMARY KEY,
        name TEXT,
        agency TEXT,
        launch_id TEXT
    )
''')

# Insert crew data into the database
for member in crew_data:
    for launch in member['launches']:
        c.execute('''
            INSERT OR IGNORE INTO crew (crew_id, name, agency, launch_id)
            VALUES (?, ?, ?, ?)
        ''', (member['id'], member['name'], member['agency'], launch))

conn.commit()
print("Crew data inserted into SQLite database.")

In [ ]:
# Define the function to fetch crew by launch ID
def get_crew_by_launch_id(launch_id):
    c.execute('''
        SELECT name, agency
        FROM crew
        WHERE launch_id = ?;
    ''', (launch_id,))
    return c.fetchall()

# Example usage of the function
sample_launch_id = spacex_data.launches[0]['id']  # Use a sample launch ID from the fetched data
crew = get_crew_by_launch_id(sample_launch_id)
print(f"Crew for launch {sample_launch_id}: {crew}")